In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from copy import deepcopy
import pandas as pnd
import time

In [217]:
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#----- Constants
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
# URLs
realtyURL = "https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/"
passportURL = 'https://passport.yandex.ru/auth?origin=realty_moscow&retpath=https%3A%2F%2Frealty.yandex.ru%2Fotsenka-kvartiry-po-adresu-onlayn%2F'

# Input file names file names
addressesReforma = 'reformagkh.csv'
addressesReformaFixed = 'reformagkh_fixed.csv'
pricesYandex = 'prices.csv'


In [285]:
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#----- Parse
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(passportURL)

# Login
login = browser.find_element_by_name('login')
login.send_keys('annastasiapb')
browser.find_element_by_xpath('/html/body/div/div/div/div[2]/div/div/div[3]/div[2]/div/div/div[1]/form/div[3]/button[1]').click()
time.sleep(2)
login = browser.find_element_by_name('passwd')
login.send_keys('Nastya200')
browser.find_element_by_xpath('/html/body/div/div/div/div[2]/div/div/div[3]/div[2]/div/div/form/div[3]/button').click()
time.sleep(2)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\1\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache


In [201]:
# Запятые - плохой сепаратор, у меня путаются запятые внутри и вне двойных кавычек, давайте заменим на что-нибудь уникальное
def change_sep(s, sep='#~#'):
    result = []
    balance = 0
    last_quotes = 0
    for i, c in enumerate(s):
        if s[i - 1:i + 1] == '""' and last_quotes != i - 1:
            balance = 1 - balance
            last_quotes = i
        if c == ',' and not balance:
            result.append(sep)
        else:
            result.append(c)
    return ''.join(result)
            

rows = []
with open(addressesReforma, encoding='utf-8') as f:
    rows.append(change_sep(f.readline())[:-1])
    for line in f.readlines():
        rows.append(change_sep(line[1:-2]))

with open(addressesReformaFixed, 'w', encoding='utf-8') as f:
    for row in rows:
        f.write(row + '\n')

data = pnd.read_csv(addressesReformaFixed, sep='#~#', encoding="utf-8")
data.head()

In [206]:
# Load ReformaGKH data
addressesList = data['address'].values.tolist()

In [286]:
# Collect data
prices = []
house_to_features = {}
for i, addressItem in enumerate(addressesList[:100]):
    print(i + 1, addressItem)
    addressInput = browser.find_element_by_name('address')
    if i == 0:
        buttonFind = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[2]/div/div[5]/div/form/div[2]/button')
    else:
        buttonFind = browser.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[2]/div/div[1]/div/form/div[2]/button')
    addressInput.send_keys(Keys.CONTROL + 'a')
    addressInput.send_keys(addressItem)
    buttonFind.click()
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    pages = soup.findAll('a', {'class': 'Pager__radio-link'})
    pages = [page['href'] for page in pages]
    pages = (['/'] if pages == [] else pages)
    current_url = browser.current_url
    for page in pages:
        if page != '/':
            browser.get(current_url + page)
            time.sleep(3)                                                                                                                  
            soup = BeautifulSoup(browser.page_source, 'lxml')
        rows = soup.findAll('div', {'class': 'OffersArchiveSearchOffers__row'})
        for row in rows[1:]:
            cells = row.findAll('div', {'class': 'OffersArchiveSearchOffers__cell'})
            for c, cell in enumerate(cells[1:]):
                if c == 0:
                    if cells[5].text == 'В продаже':
                        title = cell.find('a').text.split(', ')                    
                    else:
                        title = cell.find('div', {'class': 'OffersArchiveSearchOffers__title'}).text.split(', ')
                    extra_info = cell.find('div', {'class': 'OffersArchiveSearchOffers__extra-info'}).text
                if c == 1:
                    start_price = cell.find('div', {'class': 'OffersArchiveSearchOffers__price'}).text
                if c == 2:
                    end_price = cell.find('div', {'class': 'OffersArchiveSearchOffers__price'}).text
                if c == 3:
                    start_date = cell.text[:10]
                if c == 4:
                    end_date = cell.text    
            address_input = browser.find_element_by_name('address')
            yandex_address = address_input.get_attribute('value')

            if addressItem in house_to_features:
                features = house_to_features[addressItem]
            else:
                column = soup.findAll('div', {'class': 'BuildingInfo__feature'})
                features = []
                for cell in column:
                    features.append(cell.text)
                house_to_features[addressItem] = features
            n_features = len(features)
            features = '\t'.join(features)

            prices.append([addressItem, yandex_address] + title + [
                extra_info, start_price, end_price, start_date, end_date
            ] + [n_features, features])
    time.sleep(1)

pnd.DataFrame(prices).to_csv(pricesYandex, encoding='utf8', index=False)

1 "", 2301""
2 "", 2302""
3 "", 2303""
4 "", 2304""
5 "", 42к2""
6 "", 9к1""
7 ""1-й Амбулаторный проезд, 2/6""
8 ""1-й Амбулаторный проезд, 5к1""
9 ""1-й Амбулаторный проезд, 5к2""
10 ""1-й Амбулаторный проезд, 7к3""
11 ""1-й Амбулаторный проезд, 7кА""
12 ""1-й Амбулаторный проезд, 7кБ""
13 ""1-й Балтийский переулок, 3/25""
14 ""1-й Балтийский переулок, 4""
15 ""1-й Басманный переулок, 12""
16 ""1-й Басманный переулок, 4""
17 ""1-й Басманный переулок, 5/20с1""
18 ""1-й Басманный переулок, 5/20с2""
19 ""1-й Басманный переулок, 5/20с3""
20 ""1-й Ботанический проезд, 1""
21 ""1-й Ботанический проезд, 3""
22 ""1-й Ботанический проезд, 3А""
23 ""1-й Ботанический проезд, 5""
24 ""1-й Боткинский проезд, 2/6""
25 ""1-й Боткинский проезд, 4""
26 ""1-й Боткинский проезд, 6""
27 ""1-й Вешняковский проезд, 8""
28 ""1-й Войковский проезд, 10""
29 ""1-й Войковский проезд, 12""
30 ""1-й Войковский проезд, 14""
31 ""1-й Войковский проезд, 14Б""
32 ""1-й Войковский проезд, 14В""
33 ""1-й Войковский пр

In [287]:
pnd.DataFrame(prices)

,0,1,2,3,4,5,6,7,8,9,10
0,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",54 м²,3-комнатная,2 этаж,"8,2 млн ₽","7,2 млн ₽",11.11.2017,26.02.2018,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод
1,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",56 м²,3-комнатная,8 этаж,"9,4 млн ₽","10,5 млн ₽",08.10.2015,21.02.2017,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод
2,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",18 м²,1-комнатная,1 этаж,"3,3 млн ₽","3,3 млн ₽",25.07.2015,20.06.2016,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод
3,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",58 м²,3-комнатная,7 этаж,10 млн ₽,10 млн ₽,06.10.2015,03.12.2015,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод
4,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",42 м²,2-комнатная,8 этаж,"8,1 млн ₽","8,1 млн ₽",25.10.2015,29.10.2015,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод
...,...,...,...,...,...,...,...,...,...,...,...
1614,"""""1-й Колобовский переулок, 13/14с2""""","1-й Колобовский переулок, 13/14с2",108 м²,3-комнатная,2 этаж,75 млн ₽,75 млн ₽,05.08.2015,25.07.2016,8,3 этажа\tКирпичное здание\tЦентральное отоплен...
1615,"""""1-й Колобовский переулок, 13/14с2""""","1-й Колобовский переулок, 13/14с2",104 м²,5-комнатная,3 этаж,32 млн ₽,32 млн ₽,28.03.2016,27.04.2016,8,3 этажа\tКирпичное здание\tЦентральное отоплен...
1616,"""""1-й Колобовский переулок, 13/14с2""""","1-й Колобовский переулок, 13/14с2",46 м²,2-комнатная,1 этаж,"12,5 млн ₽","12,5 млн ₽",27.11.2014,17.04.2015,8,3 этажа\tКирпичное здание\tЦентральное отоплен...
1617,"""""1-й Колобовский переулок, 13/14с2""""","1-й Колобовский переулок, 13/14с2",68 м²,3-комнатная,2 этаж,"17,75 млн ₽","17,75 млн ₽",27.11.2014,03.03.2015,8,3 этажа\tКирпичное здание\tЦентральное отоплен...


In [ ]:
data = pnd.DataFrame(prices)
data.to_csv(pricesYandex)

In [288]:
from collections import Counter
Counter(pnd.DataFrame(prices)[9])

Counter({4: 43, 7: 969, 8: 234, 6: 321, 5: 52})

In [289]:
all_features = [s.split('\t') for s in data.iloc[:, 10]]

In [290]:
import numpy as np
Counter(np.concatenate(all_features)).most_common()

[('Кирпичное здание', 1273),
 ('Лифт', 1043),
 ('Газ', 929),
 ('Мусоропровод', 925),
 ('Центральное отопление', 906),
 ('Мусоропровода нет', 625),
 ('Лифт отсутствует', 565),
 ('5\xa0этажей', 541),
 ('3,00\xa0м потолки', 526),
 ('Газа нет', 474),
 ('3,20\xa0м потолки', 345),
 ('2,48\xa0м потолки', 218),
 ('7\xa0этажей', 176),
 ('Монолитное здание', 147),
 ('8\xa0этажей', 121),
 ('43\xa0квартиры', 120),
 ('96\xa0квартир', 120),
 ('11\xa0этажей', 117),
 ('80\xa0квартир', 111),
 ('60\xa0квартир', 106),
 ('Панельное здание', 98),
 ('6\xa0этажей', 96),
 ('10\xa0этажей', 90),
 ('2,64\xa0м потолки', 76),
 ('3,40\xa0м потолки', 69),
 ('189\xa0квартир', 67),
 ('4\xa0этажа', 66),
 ('139\xa0квартир', 63),
 ('3\xa0этажа', 61),
 ('12\xa0этажей', 51),
 ('3,10\xa0м потолки', 46),
 ('76\xa0квартир', 37),
 ('123\xa0квартиры', 36),
 ('159\xa0квартир', 35),
 ('183\xa0квартиры', 34),
 ('70\xa0квартир', 32),
 ('40\xa0квартир', 31),
 ('2,70\xa0м потолки', 29),
 ('116\xa0квартир', 29),
 ('111\xa0квартир', 29

In [291]:
feature_dict = {
    'Лифт': ('has_elevator', 1),
    'Лифт отсутствует': ('has_elevator', 0),
    'Панельное здание': ('material', 'Панельное здание'),
    'Кирпичное здание': ('material', 'Кирпичное здание'),
    'Монолитное здание': ('material', 'Монолитное здание'),
    'Кирпично-монолитное здание': ('material', 'Кирпично-монолитное здание'),
    'Блочное здание': ('material', 'Блочное здание'),
    'Газ': ('has_gas', 1),
    'Газа нет': ('has_gas', 0),
    'Мусоропровод': ('has_garbage', 1),
    'Мусоропровода нет': ('has_garbage', 0),
    'Центральное отопление': ('heating', 'Центральное отопление'),
}

In [292]:
from copy import copy
import re

def get_number(text):
    pattern = re.compile("(\d+\,?\d*)")
    number = pattern.search(text).groups()[0].replace(',', '.')
    if '.' in number:
        return float(number)
    return int(number)

new_rows = []
for id, row in data.iterrows():
    new_row = copy(row)
    for feature in row[10].split('\t'):
        try:
            if feature in feature_dict:
                feature_name, feature_value = feature_dict[feature]
                new_row[feature_name] = feature_value
            elif 'квартир' in feature:
                new_row['flats'] = get_number(feature)
            elif 'этаж' in feature:
                new_row['floors'] = get_number(feature)
            elif 'потол' in feature:
                new_row['ceiling'] = get_number(feature)
        except Exception as e:
            print('Error in feature =', feature)
            print(e)
    new_rows.append(new_row)
new_data = pnd.DataFrame(new_rows)
new_data.head(20)

,0,1,2,3,4,5,6,7,8,9,10,floors,material,has_elevator,has_garbage,heating,flats,has_gas,ceiling
0,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",54 м²,3-комнатная,2 этаж,"8,2 млн ₽","7,2 млн ₽",11.11.2017,26.02.2018,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод,9.0,Кирпичное здание,1.0,1.0,NaN,NaN,NaN,NaN
1,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",56 м²,3-комнатная,8 этаж,"9,4 млн ₽","10,5 млн ₽",08.10.2015,21.02.2017,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод,9.0,Кирпичное здание,1.0,1.0,NaN,NaN,NaN,NaN
2,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",18 м²,1-комнатная,1 этаж,"3,3 млн ₽","3,3 млн ₽",25.07.2015,20.06.2016,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод,9.0,Кирпичное здание,1.0,1.0,NaN,NaN,NaN,NaN
3,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",58 м²,3-комнатная,7 этаж,10 млн ₽,10 млн ₽,06.10.2015,03.12.2015,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод,9.0,Кирпичное здание,1.0,1.0,NaN,NaN,NaN,NaN
4,"""""1-й Амбулаторный проезд, 2/6""""","1-й Амбулаторный проезд, вл2-6",42 м²,2-комнатная,8 этаж,"8,1 млн ₽","8,1 млн ₽",25.10.2015,29.10.2015,4,9 этажей\tКирпичное здание\tЛифт\tМусоропровод,9.0,Кирпичное здание,1.0,1.0,NaN,NaN,NaN,NaN
5,"""""1-й Амбулаторный проезд, 5к1""""","1-й Амбулаторный проезд, 5к1",42 м²,2-комнатная,2 этаж,"9,25 млн ₽","8,5 млн ₽",06.03.2020,В продаже,7,5 этажей\tКирпичное здание\tЦентральное отопле...,5.0,Кирпичное здание,0.0,0.0,Центральное отопление,60.0,1.0,NaN
6,"""""1-й Амбулаторный проезд, 5к1""""","1-й Амбулаторный проезд, 5к1","43,5 м²",2-комнатная,5 этаж,"7,7 млн ₽","7,5 млн ₽",29.06.2018,13.10.2019,7,5 этажей\tКирпичное здание\tЦентральное отопле...,5.0,Кирпичное здание,0.0,0.0,Центральное отопление,60.0,1.0,NaN
7,"""""1-й Амбулаторный проезд, 5к1""""","1-й Амбулаторный проезд, 5к1",44 м²,2-комнатная,3 этаж,"10,4 млн ₽","10,4 млн ₽",08.08.2019,11.09.2019,7,5 этажей\tКирпичное здание\tЦентральное отопле...,5.0,Кирпичное здание,0.0,0.0,Центральное отопление,60.0,1.0,NaN
8,"""""1-й Амбулаторный проезд, 5к1""""","1-й Амбулаторный проезд, 5к1",42 м²,2-комнатная,2 этаж,"8,1 млн ₽","8,1 млн ₽",19.07.2017,26.11.2018,7,5 этажей\tКирпичное здание\tЦентральное отопле...,5.0,Кирпичное здание,0.0,0.0,Центральное отопление,60.0,1.0,NaN
9,"""""1-й Амбулаторный проезд, 5к1""""","1-й Амбулаторный проезд, 5к1","43,1 м²",2-комнатная,1 этаж,"7,6 млн ₽","7,2 млн ₽",07.09.2018,05.11.2018,7,5 этажей\tКирпичное здание\tЦентральное отопле...,5.0,Кирпичное здание,0.0,0.0,Центральное отопление,60.0,1.0,NaN


In [293]:
new_data.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,floors,material,has_elevator,has_garbage,heating,flats,has_gas,ceiling
1599,"""""1-й Колобовский переулок, 10с1""""","1-й Колобовский переулок, 10с1",82 м²,3-комнатная,3 этаж,32 млн ₽,32 млн ₽,04.02.2017,12.03.2017,7,6 этажей\tКирпичное здание\tЦентральное отопле...,6.0,Кирпичное здание,1.0,0.0,Центральное отопление,16.0,1.0,NaN
1600,"""""1-й Колобовский переулок, 10с1""""","1-й Колобовский переулок, 10с1",70 м²,3-комнатная,3 этаж,28 млн ₽,28 млн ₽,27.11.2014,28.06.2015,7,6 этажей\tКирпичное здание\tЦентральное отопле...,6.0,Кирпичное здание,1.0,0.0,Центральное отопление,16.0,1.0,NaN
1601,"""""1-й Колобовский переулок, 10с1""""","1-й Колобовский переулок, 10с1",136 м²,3-комнатная,4 этаж,"74,7 млн ₽","74,7 млн ₽",06.12.2014,03.03.2015,7,6 этажей\tКирпичное здание\tЦентральное отопле...,6.0,Кирпичное здание,1.0,0.0,Центральное отопление,16.0,1.0,NaN
1602,"""""1-й Колобовский переулок, 11""""","1-й Колобовский переулок, 11",22 м²,студия,2 этаж,"7,9 млн ₽","7,9 млн ₽",22.06.2020,В продаже,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,NaN,1.0,3.0
1603,"""""1-й Колобовский переулок, 11""""","1-й Колобовский переулок, 11",54 м²,3-комнатная,1 этаж,17 млн ₽,15 млн ₽,27.11.2014,28.09.2015,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,NaN,1.0,3.0
1604,"""""1-й Колобовский переулок, 11""""","1-й Колобовский переулок, 11",43 м²,1-комнатная,5 этаж,"12,8 млн ₽","12,8 млн ₽",02.07.2015,13.07.2015,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,NaN,1.0,3.0
1605,"""""1-й Колобовский переулок, 11""""","1-й Колобовский переулок, 11",40 м²,2-комнатная,2 этаж,15 млн ₽,"14,3 млн ₽",11.03.2015,08.07.2015,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,NaN,1.0,3.0
1606,"""""1-й Колобовский переулок, 11""""","1-й Колобовский переулок, 11",43 м²,1-комнатная,5 этаж,"14,8 млн ₽","14,8 млн ₽",26.02.2015,08.05.2015,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,NaN,1.0,3.0
1607,"""""1-й Колобовский переулок, 13/14/2с4""""","1-й Колобовский переулок, 13/14/2с4",104 м²,студия,3 этаж,"4,2 млн ₽","3,95 млн ₽",22.08.2016,30.10.2016,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,6.0,0.0,NaN
1608,"""""1-й Колобовский переулок, 13/14/2с4""""","1-й Колобовский переулок, 13/14/2с4",170 м²,4-комнатная,2 этаж,132 млн ₽,132 млн ₽,27.05.2016,19.07.2016,7,3 этажа\tКирпичное здание\tЦентральное отоплен...,3.0,Кирпичное здание,0.0,0.0,Центральное отопление,6.0,0.0,NaN
